In [ ]:
#!pip install snowflake-connector-python

In [1]:
import snowflake.connector
import pandas as pd
import json

'''user = REFRACT.FOSFOR@LNTINFOTECH.COM
password = Password321
account = fya62509.us-east-1
warehouse = FOSFOR_REFRACT
database = FOSFOR_REFRACT
schema = SALES
role = FOSFOR_REFRACT'''    

# Snowflake connection parameters
snowflake_account = 'rwb76633'
snowflake_user = 'Ravi'
snowflake_password = 'Kayakave@1983'
snowflake_database = 'TUTORIAL_DB'
snowflake_schema = 'DATA_SCHEMA'
snowflake_stage = 'MAGICODER2'
role = 'TEST_ROLE'

# snowflake_account = 'fya62509.us-east-1'
# snowflake_user = 'REFRACT.FOSFOR@LNTINFOTECH.COM'
# snowflake_password = 'Password321'
# snowflake_database = 'FOSFOR_REFRACT'
# snowflake_schema = 'SALES'
# snowflake_stage = 'MAGICODER'
 
# Connect to Snowflake
conn = snowflake.connector.connect(
   user=snowflake_user,
   password=snowflake_password,
   account=snowflake_account,
   session_parameters={
      'PYTHON_CONNECTOR_QUERY_RESULT_FORMAT': 'json'
   },
   database=snowflake_database,
   schema=snowflake_schema,
   role=role,
)




In [2]:
conn

In [3]:
#! ls -lrt /llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66

In [4]:
# Create a cursor object
cur = conn.cursor()
 
# Upload file to stage (example using Pandas)
#file_path = '/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66/special_tokens_map.json'
# df = pd.read_csv(file_path)
#cur.execute(f"PUT file://{file_path} @{snowflake_stage}/magicoder_model AUTO_COMPRESS = FALSE")
#cur.execute("Select * from Sales")

In [16]:
prompt= """You are an exceptional dbt analytics engineer writes dbt models, also creates very precise and accurate functional documentation to ensure any layman understands the code written by you.
        follow instruction and response accuratly. finally only generate respose without instruction. 
        @@Instruction
        this is the sample dbt model
        with customer_revenue as (
        select
        customer_id,
        sum(amount) as total_revenue
        from
        {{ ref('orders') }}
        group by 1
        )
        select
        c.customer_id,
        c.first_name,
        c.last_name,
        cr.total_revenue
        from
        {{ source('master_data','customers') }} c
        left join customer_revenue cr on c.customer_id = cr.customer_id
 
        @@Response
        Below is the sample dbt model documentation generated for above instruction dbt model:
        * Overview: This dbt model calculates the total revenue per customer by leveraging the data from the orders model and customers source. 
        * Dependencies:
        * orders: dbt model that contains information about individual orders, including the customer_id and amount.
        * customers: dbt source contains customer details, including customer_id, first_name, and last_name.
        * Transformation:
        * The CTE 'customer_revenue' aggregates the total revenue for each customer by summing the amount from the orders model. It utilizes the customer_id field for grouping.
        * The final output includes customer details from the customers table (c) and their corresponding total revenue from the customer_revenue CTE (cr). The join is performed using the customer_id field.
        * Example Usage:	The resulting model can be used for various analytical purposes, such as identifying high-value customers or generating reports on customer revenue.
 
        @@ Instruction
        create similar dbt model documentation for following dbt model code
            "create document for following dbt code {% snapshot users_snapshot %}  
 
    {{
      config(      
        target_schema='snapshots',      
        strategy='timestamp',      
        unique_key='id',      
        updated_at='updated_at'    
      )  
    }}  
 
      select * 
      from {{ source('raw','users') }}
 
    {% endsnapshot %}"""


In [6]:
#prompt = r"""Construct a DBT view model in the 'staging' schema, linking 'jaffle_shop' orders with payment information from 'stg_payments', summarizing successful payments per order, and merging with order details to handle potential missing payment scenarios."""

In [17]:
prompt = prompt.replace("'", "''")
prompt = prompt.replace("\n", " ")

In [18]:
arg = '{"prompt":'+'"'+prompt+'"'+', "max_tokens": 2000, "temperature": 0, "model": "/data/magicoder_model"}'


In [19]:
import json
arg = json.dumps(arg)

In [20]:
arg

'"{\\"prompt\\":\\"You are an exceptional dbt analytics engineer writes dbt models, also creates very precise and accurate functional documentation to ensure any layman understands the code written by you.         follow instruction and response accuratly. finally only generate respose without instruction.          @@Instruction         this is the sample dbt model         with customer_revenue as (         select         customer_id,         sum(amount) as total_revenue         from         {{ ref(\'\'orders\'\') }}         group by 1         )         select         c.customer_id,         c.first_name,         c.last_name,         cr.total_revenue         from         {{ source(\'\'master_data\'\',\'\'customers\'\') }} c         left join customer_revenue cr on c.customer_id = cr.customer_id           @@Response         Below is the sample dbt model documentation generated for above instruction dbt model:         * Overview: This dbt model calculates the total revenue per customer by

In [21]:
arg = '{"prompt":'+'"'+prompt+'"'+', "max_tokens": 2000, "temperature": 0, "model": "/data/magicoder_model"}'
warehouse = 'USE WAREHOUSE TUTORIAL_WAREHOUSE;'
query = f"SELECT magicoder_udf(parse_json('{arg}'));"
print(query)

SELECT magicoder_udf(parse_json('{"prompt":"You are an exceptional dbt analytics engineer writes dbt models, also creates very precise and accurate functional documentation to ensure any layman understands the code written by you.         follow instruction and response accuratly. finally only generate respose without instruction.          @@Instruction         this is the sample dbt model         with customer_revenue as (         select         customer_id,         sum(amount) as total_revenue         from         {{ ref(''orders'') }}         group by 1         )         select         c.customer_id,         c.first_name,         c.last_name,         cr.total_revenue         from         {{ source(''master_data'',''customers'') }} c         left join customer_revenue cr on c.customer_id = cr.customer_id           @@Response         Below is the sample dbt model documentation generated for above instruction dbt model:         * Overview: This dbt model calculates the total revenue pe

In [22]:
cur.execute(warehouse)
cur.execute(query)
result = cur.fetchall()
print(json.loads(result[0][0])['choices'][0]['text'])

ProgrammingError: 100069 (22P02): Error parsing JSON: missing comma, pos 1989

In [ ]:
result

In [23]:
import requests
import json

# Obtain a session token.
token_data = conn._rest._token_request('ISSUE')
token_extract = token_data['data']['sessionToken']

# Create a request to the ingress endpoint with authz.
token = f'\"{token_extract}\"'
# Set this to the ingress endpoint URL for your service
# old_url = 'http://gowiocnz-zqiseam-ltifosforscsaws.snowflakecomputing.app/vllm/magicoder/v1/completions'
url = 'http://gowiocpf-zqiseam-ltifosforscsaws.snowflakecomputing.app/vllm/magicoder/v1/models'

In [24]:
headers = {
  'Content-Type': 'application/json', 'Authorization': f'Snowflake Token={token}'
}
print(headers)
print(url)
#print(payload)

{'Content-Type': 'application/json', 'Authorization': 'Snowflake Token="ver:1-hint:14544919633592326-ETMsDgAAAY5bQ+maABRBRVMvQ0JDL1BLQ1M1UGFkZGluZwEAABAAEOyFZaxZV1FRemtKZbDTHaQAAACAdBZEu9EZw1hVf3vnKJ2bgh1SlgRUDndhKBNLogf23dopn6HVJX8TzFAjvLTY+skQnT0UIHkVhOcTZYOpDudxKUdK/AbiOjc46YB9C6IwmNmfHb9rKBI/tfjiGy7kozwmnrSEcvQlUMlCXGnQWPZdO2iRcF+IcGzH5pX3Q7C+JXIAFJ0YcZmKWWL+QcBw9uMp68HxJCqC"'}
http://gowiocpf-zqiseam-ltifosforscsaws.snowflakecomputing.app/vllm/magicoder/v1/models


In [25]:
payload = json.dumps({
  "prompt": "write a python code to check whether given string is pallindrome",
  "max_tokens": 2400,
  "temperature": 0,
  "model": "/data/magicoder_model"
})
headers = {
  'Content-Type': 'application/json', 'Authorization': f'Snowflake Token={token}'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


ConnectionError: HTTPConnectionPool(host='gowiocpf-zqiseam-ltifosforscsaws.snowflakecomputing.app', port=80): Max retries exceeded with url: /vllm/magicoder/v1/models (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2873c22d30>: Failed to establish a new connection: [Errno -2] Name or service not known'))